In [2]:
import huggingface_hub
huggingface_hub.login()

In [1]:
#@title Installation
from importlib.util import find_spec
in_colab = find_spec("google.colab") is not None


src = """[tool.uv]
override-dependencies = ["""
replace_with = """
    "torch ; sys_platform == 'never'",
    "pytorch-triton ; sys_platform == 'never'",
"""

if in_colab:
    %env SPARSIFY_DISABLE_TRITON=1
    %env OFFLOAD_TRANSCODER=1
    %cd /content
    !git clone https://github.com/EleutherAI/attribute
    %cd attribute
    !git pull
    pyproject_text = open("pyproject.toml").read()
    if src in pyproject_text:
        after_src = pyproject_text.index(src) + len(src)
        current = pyproject_text[after_src:][:len(replace_with)]
        if current != replace_with:
            pyproject_text = pyproject_text[:after_src] + replace_with + pyproject_text[after_src:]
            open("pyproject.toml", "w").write(pyproject_text)
    !curl -LsSf https://astral.sh/uv/install.sh | sh
    !uv pip install -e .

In [ ]:
!mkdir -p results && cd results && git clone https://huggingface.co/nev/Llama-3.2-1B-mntss-skip-transcoder-cache --depth=1

In [ ]:
import serve
import torch
torch._functorch.config.donated_buffer = False
serve.default_config.batch_size = 8
serve.main().launch(share=True, debug=True)